## Alejandro Pulido Sánchez

# PRÁCTICA 8: REGRESIÓN LINEAL MÚLTIPLE Y LOGÍSTICA

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sys import float_info

## Regresión Lineal Múltiple

<div class="alert alert-block alert-success">
<b>La línea X = MinMaxScaler().fit_transform(datos.iloc[:, :-1])</b> escala todas las características del conjunto de datos, excepto la variable de respuesta, utilizando la técnica de escalamiento Min-Max, que transforma los valores de las características en el rango [0, 1]. El objeto MinMaxScaler() se ajusta y se transforma al conjunto de datos utilizando el método fit_transform().

<b>La línea Yb = LabelBinarizer().fit_transform(datos.iloc[:, -1])</b> transforma la variable de respuesta en una matriz de valores binarios utilizando la técnica de binarización de etiquetas "onehot". El objeto LabelBinarizer() se ajusta y se transforma a la variable de respuesta utilizando el método fit_transform().

<b>La línea Y = datos.iloc[:, -1]</b> selecciona la variable de respuesta original sin transformar y la asigna a la variable Y

In [2]:
datos = pd.read_csv("dataset_36_segment.csv")
X = MinMaxScaler().fit_transform(datos.iloc[:, :-1])
Yb = LabelBinarizer().fit_transform(datos.iloc[:, -1]) #Binario "onehot"
Y = datos.iloc[:, -1]
datos

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean,class
0,218,178,9,0.111111,0.0,0.833333,0.547722,1.111110,0.544331,59.629600,52.44440,75.22220,51.222200,-21.555600,46.77780,-25.22220,75.22220,0.318996,-2.04055,path
1,113,130,9,0.000000,0.0,0.277778,0.250924,0.333333,0.365148,0.888889,0.00000,2.55556,0.111111,-2.666670,5.00000,-2.33333,2.55556,1.000000,-2.12325,foliage
2,202,41,9,0.000000,0.0,0.944448,0.772202,1.111110,1.025600,123.037000,111.88900,139.77800,117.444000,-33.444400,50.22220,-16.77780,139.77800,0.199347,-2.29992,sky
3,32,173,9,0.000000,0.0,1.722220,1.781590,9.000000,6.749490,43.592600,39.55560,52.88890,38.333300,-12.111100,27.88890,-15.77780,52.88890,0.266914,-1.99886,path
4,61,197,9,0.000000,0.0,1.444440,1.515350,2.611110,1.925460,49.592600,44.22220,61.55560,43.000000,-16.111100,35.88890,-19.77780,61.55560,0.302925,-2.02227,path
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,30,102,9,0.000000,0.0,1.222220,0.118518,1.333330,0.800000,20.259300,20.33330,25.00000,15.444400,0.222222,14.22220,-14.44440,25.00000,0.381059,-1.55510,brickface
2306,143,24,9,0.000000,0.0,1.277780,0.907406,0.888888,1.140750,127.630000,117.66700,141.66700,123.556000,-29.888900,42.11110,-12.22220,141.66700,0.169397,-2.34925,sky
2307,80,72,9,0.000000,0.0,1.222220,1.003700,1.444440,1.167460,59.000000,51.33330,74.44440,51.222200,-23.000000,46.33330,-23.33330,74.44440,0.314606,-2.09022,cement
2308,98,133,9,0.000000,0.0,0.555555,0.172133,0.388889,0.327731,0.962963,0.00000,2.77778,0.111111,-2.888890,5.44444,-2.55556,2.77778,1.000000,-2.12325,foliage


<div class="alert alert-block alert-success">
<b>train_test_split(X, Yb, test_size=1/3, stratify=Yb):</b> divide el conjunto de datos en conjuntos de entrenamiento y prueba (X_train, X_test, Y_train, Y_test) en una proporción del 2:1 (un tercio para el conjunto de prueba) utilizando el método de remuestreo aleatorio. X es el conjunto de características transformadas y escaladas y Yb es el conjunto de variables de respuesta transformadas. El argumento test_size especifica la proporción del conjunto de prueba y stratify=Yb asegura que la división del conjunto de datos sea estratificada, es decir, que la proporción de clases en el conjunto de entrenamiento y prueba sea la misma que en el conjunto de datos original.

<b>X_train.shape, X_test.shape, Y_train.shape, Y_test.shape:</b> imprime las dimensiones de los conjuntos de entrenamiento y prueba. En particular, X_train.shape y Y_train.shape devuelven el número de filas y columnas en los conjuntos de entrenamiento para las características y variables de respuesta, respectivamente, mientras que X_test.shape y Y_test.shape devuelven el número de filas y columnas en los conjuntos de prueba para las características y variables de respuesta, respectivamente.

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Yb, test_size=1/3, stratify=Yb)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1540, 19), (770, 19), (1540, 7), (770, 7))

<div class="alert alert-block alert-success">
<b>Y_predict = float_info.min*np.ones(Y_test.shape, dtype=float):</b> crea una matriz Y_predict con el mismo número de filas y columnas que Y_test y la inicializa con el valor mínimo posible de un número de punto flotante, obtenido del módulo sys.float_info, multiplicado por un vector de unos. El argumento dtype=float asegura que la matriz sea de tipo de datos de punto flotante.

<b>Y_predict:</b> devuelve la matriz Y_predict. Esta matriz se utiliza comúnmente en la evaluación de modelos de aprendizaje automático para inicializar las predicciones antes de ajustar el modelo y predecir valores reales.

In [4]:
Y_predict = float_info.min*np.ones(Y_test.shape, dtype=float)
Y_predict

array([[2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       ...,
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308, ...,
        2.22507386e-308, 2.22507386e-308, 2.22507386e-308]])

<div class="alert alert-block alert-success">
<b>for i in range (Y_train.shape[1]):</b> itera a través de todas las variables de respuesta en el conjunto de entrenamiento, que corresponden a las columnas de Y_train, utilizando el método range().

<b>model = LinearRegression():</b> crea una instancia del modelo de regresión lineal de la biblioteca sklearn.linear_model.

<b>model.fit(X_train, Y_train[:,i]):</b> ajusta el modelo a los datos de entrenamiento (X_train y la columna i de Y_train) utilizando el método fit() del modelo.

<b>Y_predict[:,i] = model.predict(X_test):</b> utiliza el modelo ajustado para hacer predicciones en el conjunto de prueba (X_test) y las asigna a la columna i de Y_predict.

<b>Y_test.shape[0]:</b> devuelve el número de filas en el conjunto de prueba Y_test.

In [5]:
for i in range (Y_train.shape[1]):
    model = LinearRegression()
    model.fit(X_train, Y_train[:,i])
    Y_predict[:,i] = model.predict(X_test)
Y_test.shape[0]

770

<div class="alert alert-block alert-success">
<b>np.argmax(Y_test,axis=1):</b> devuelve la posición del valor máximo en cada fila del conjunto de prueba Y_test. Esto se utiliza para convertir las etiquetas binarias codificadas en un vector de etiquetas.

<b>np.argmax(Y_predict, axis=1):</b> devuelve la posición del valor máximo en cada fila de la matriz Y_predict. Esto se utiliza para convertir las etiquetas binarias codificadas en un vector de etiquetas.

<b>accuracy_score(np.argmax(Y_test,axis=1), np.argmax(Y_predict, axis=1)):</b> utiliza la función accuracy_score de la biblioteca sklearn.metrics para calcular la tasa de aciertos entre el vector de etiquetas reales (np.argmax(Y_test,axis=1)) y el vector de etiquetas predichas (np.argmax(Y_predict, axis=1)). La tasa de aciertos se define como el número de predicciones correctas dividido por el número total de predicciones.

In [6]:
print("Tasa de aciertos =",accuracy_score(np.argmax(Y_test,axis=1), np.argmax(Y_predict, axis=1)))

Tasa de aciertos = 0.8402597402597403


<div class="alert alert-block alert-success">
<b>confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)):</b> utiliza la función confusion_matrix de la biblioteca sklearn.metrics para calcular la matriz de confusión entre el vector de etiquetas reales (np.argmax(Y_test, axis=1)) y el vector de etiquetas predichas (np.argmax(Y_predict, axis=1)). La matriz de confusión es una tabla que muestra el número de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos para cada clase.

En conjunto, este fragmento de código calcula y muestra la matriz de confusión del modelo utilizando las etiquetas reales y predichas del conjunto de prueba. La matriz de confusión proporciona información detallada sobre el rendimiento del modelo para cada clase.






In [7]:
print("\nMatriz de Confusión:\n", confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)))


Matriz de Confusión:
 [[107   0   3   0   0   0   0]
 [ 24  42   2   0  27   2  13]
 [  0   5  95   0   0   0  10]
 [  0   0   0 108   2   0   0]
 [  0   0   0   0 110   0   0]
 [  0   0   0   0   0 110   0]
 [  1   3  29   0   2   0  75]]


## Regresion Logistica

In [8]:
precision = 0
K = 10
kfold = StratifiedKFold(n_splits=K)

<div class="alert alert-block alert-success">
Este fragmento de código realiza validación cruzada utilizando StratifiedKFold para dividir el conjunto de datos en 10 pliegues y calcular la tasa de error promedio del modelo en cada pliegue. En resumen:

<b>cv = StratifiedKFold(n_splits=10):</b> crea una instancia del objeto StratifiedKFold con 10 pliegues.

<b>for train_index, test_index in cv.split(X,Y)::</b> divide el conjunto de datos en 10 pliegues utilizando cv.split(), que devuelve los índices de los datos de entrenamiento y prueba en cada pliegue.
    
<b>X_train, X_test = X[train_index], X[test_index]:</b> separa los datos de entrada en conjuntos de entrenamiento y prueba basados en los índices de los pliegues actuales.

<b>Y_train, Y_test = Yb[train_index], Yb[test_index]:</b> separa las etiquetas binarias en conjuntos de entrenamiento y prueba basados en los índices de los pliegues actuales.

<b>for i in range(Y_train.shape[1])::</b> itera a través de las etiquetas de cada clase.

<b>linReg = LinearRegression():</b> crea una instancia de LinearRegression.

<b>linReg.fit(X_train, Y_train[:,i]):</b> entrena el modelo de regresión lineal utilizando los datos de entrada y las etiquetas de la clase actual.

<b>Y_predict[:,i] = linReg.predict(X_test):</b> predice las etiquetas de la clase actual utilizando los datos de prueba y el modelo entrenado.

<b>np.sum(np.argmax(Y_predict, axis=1) == np.argmax(Y_test, axis=1))/Y_test.shape[0]:</b> calcula la tasa de error para el pliegue actual, que es la fracción de predicciones incorrectas en el conjunto de prueba.

<b>precision += np.sum(np.argmax(Y_predict, axis=1) == np.argmax(Y_test, axis=1))/Y_test.shape[0]:</b> suma la tasa de error del pliegue actual a la tasa de error total.

<b>precision/10:</b> calcula la tasa de error promedio del modelo utilizando la tasa de error total y el número de pliegues.

In [9]:
precision = 0
cv = StratifiedKFold(n_splits=10)
print("Las tasas de error de las 10 carpetas son: \n")
for train_index, test_index in cv.split(X,Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Yb[train_index], Yb[test_index]
    Y_predict = np.zeros(Y_test.shape)
    for i in range(Y_train.shape[1]):
        linReg = LinearRegression()
        linReg.fit(X_train, Y_train[:,i])
        Y_predict[:,i] = linReg.predict(X_test)
    print(np.sum(np.argmax(Y_predict, axis=1) == np.argmax(Y_test, axis=1))/Y_test.shape[0])
    precision += np.sum(np.argmax(Y_predict, axis=1) == np.argmax(Y_test, axis=1))/Y_test.shape[0]

print("\nLa media de las tasas de error: ", precision/10)

Las tasas de error de las 10 carpetas son: 

0.8917748917748918
0.8225108225108225
0.8441558441558441
0.8354978354978355
0.8701298701298701
0.8528138528138528
0.8225108225108225
0.8354978354978355
0.8311688311688312
0.8571428571428571

La media de las tasas de error:  0.8463203463203464


## Ejercicio 4

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 1./3., random_state=0, stratify=Y)

<div class="alert alert-block alert-success">
<b>random_state=0</b> se utiliza para garantizar que el modelo se ajuste de la misma manera cada vez que se ejecute el código.
    
<b>solver='lbfgs'</b> es un algoritmo numérico que se utiliza para optimizar los parámetros del modelo. El algoritmo L-BFGS es una técnica de optimización que utiliza una aproximación de la matriz Hessiana inversa para actualizar los parámetros del modelo.
    
<b>multi_class='auto'</b> especifica el método utilizado para resolver problemas de clasificación multiclase. El valor "auto" ajusta el método de acuerdo con los datos.
    
<b>max_iter=500</b> establece el número máximo de iteraciones para el algoritmo de optimización utilizado por el modelo. Si la convergencia no se alcanza en ese número de iteraciones, el modelo devuelve una advertencia.
fit(X_train, y_train) ajusta el modelo a los datos de entrenamiento X_train e y_train.

In [11]:
regresion_logistica = LogisticRegression(random_state = 0, solver='lbfgs',multi_class='auto', max_iter= 500)
regresion_logistica.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=0)

In [12]:
precision = regresion_logistica.score(X_test, y_test)

<div class="alert alert-block alert-success">
En este código, <b>predict()</b> es un método de la clase LogisticRegression que se utiliza para hacer predicciones en el conjunto de datos de prueba X_test. Toma como entrada X_test y devuelve las predicciones correspondientes.

confusion_matrix() es una función de la biblioteca scikit-learn que se utiliza para calcular la matriz de confusión a partir de las verdaderas etiquetas (y_test) y las predicciones (x_pred). La matriz de confusión es una tabla que se utiliza para evaluar el rendimiento del modelo de clasificación. Muestra el número de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos.

In [13]:
x_pred = regresion_logistica.predict(X_test)
matrizconfusion = confusion_matrix(y_test,x_pred)
print("\nMatriz de Confusión:\n", matrizconfusion)


Matriz de Confusión:
 [[105   0   0   0   0   0   5]
 [  2  94   2   0   8   0   4]
 [  5   5  87   0   0   0  13]
 [  0   0   0 110   0   0   0]
 [  0   0   0   0 110   0   0]
 [  0   0   0   0   0 110   0]
 [  1   4  32   0   0   0  73]]


### VALIDACION CRUZADA

<div class="alert alert-block alert-success">
<b>En este código se crea un objeto de la clase LogisticRegression</b> con los siguientes parámetros:

random_state=0: establece la semilla aleatoria para garantizar que los resultados sean reproducibles.
multi_class='auto': especifica que el modelo debe utilizar la estrategia "one-vs-rest" para problemas de clasificación con más de dos clases.
solver='lbfgs': especifica el algoritmo a utilizar para resolver el problema de optimización subyacente.
max_iter=500: establece el número máximo de iteraciones que el algoritmo de optimización debe ejecutar antes de detenerse.

In [14]:
reg_log = LogisticRegression(random_state=0, multi_class='auto', solver='lbfgs', max_iter=500)

<div class="alert alert-block alert-success">
<b>El código realiza la validación cruzada estratificada de una regresión logística en un conjunto de datos dado por las matrices X e Y.</b> 
Se utiliza una tasa de aciertos para evaluar el rendimiento del modelo en cada una de las 10 carpetas. En cada iteración del bucle for, se dividen los datos en un conjunto de entrenamiento y un conjunto de prueba utilizando la función cv.split(). Luego, se ajusta un modelo de regresión logística a los datos de entrenamiento y se evalúa su rendimiento en los datos de prueba utilizando la función score(). La tasa de aciertos de cada iteración se suma y se calcula la tasa media de aciertos. Además, se imprime la tasa de aciertos de cada iteración y la tasa media de aciertos.

In [15]:
tasa = 0.
cv = StratifiedKFold(n_splits=10)
i = 0
for train_index, test_index in cv.split(X,Y):
    
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    reg_log.fit(X_train, Y_train)
    tasa += reg_log.score(X_test, Y_test)  # Actualiza la tasa de aciertos
    print("La tasa para la carpeta ",i," tiene una tasa de acierto de ",reg_log.score(X_test, Y_test))
    i = i+1
print("\nTasa de acierto: \n",tasa/10)

La tasa para la carpeta  0  tiene una tasa de acierto de  0.9307359307359307
La tasa para la carpeta  1  tiene una tasa de acierto de  0.9090909090909091
La tasa para la carpeta  2  tiene una tasa de acierto de  0.9047619047619048
La tasa para la carpeta  3  tiene una tasa de acierto de  0.9004329004329005
La tasa para la carpeta  4  tiene una tasa de acierto de  0.9047619047619048
La tasa para la carpeta  5  tiene una tasa de acierto de  0.9177489177489178
La tasa para la carpeta  6  tiene una tasa de acierto de  0.8961038961038961
La tasa para la carpeta  7  tiene una tasa de acierto de  0.8917748917748918
La tasa para la carpeta  8  tiene una tasa de acierto de  0.9134199134199135
La tasa para la carpeta  9  tiene una tasa de acierto de  0.8744588744588745

Tasa de acierto: 
 0.9043290043290044
